In [1]:
import os
import uuid
from decimal import Decimal

from dotenv import load_dotenv

from tinkoff.invest import OrderDirection, OrderType
from tinkoff.invest.async_services import (
    PostOrderAsyncRequest,
    PostOrderAsyncResponse,
    PriceType,
    TimeInForceType,
)
from tinkoff.invest.sandbox.async_client import AsyncSandboxClient
from tinkoff.invest.utils import decimal_to_quotation

load_dotenv()


TOKEN = os.environ["TINKOFF_TOKEN"]

In [2]:
async def get_account_id():
    async with AsyncSandboxClient(TOKEN) as client:
        response = await client.users.get_accounts()
        account, *_ = response.accounts
        account_id = account.id
        return account_id


account_id = await get_account_id()
account_id

'6def19bc-ad19-486f-92fa-090c72e531f0'

In [3]:
async def get_orders():
    async with AsyncSandboxClient(TOKEN) as client:
        orders = await client.orders.get_orders(account_id=account_id)
    return orders


orders = await get_orders()
orders

GetOrdersResponse(orders=[])

In [4]:
async def create_new_order():
    async with AsyncSandboxClient(TOKEN) as client:
        order_id = str(uuid.uuid4())
        request = PostOrderAsyncRequest(
            instrument_id="BBG0013HRTL0",
            quantity=1,
            price=decimal_to_quotation(Decimal(10.836499999)),
            direction=OrderDirection.ORDER_DIRECTION_BUY,
            account_id=account_id,
            order_type=OrderType.ORDER_TYPE_MARKET,
            order_id=order_id,
            time_in_force=TimeInForceType.TIME_IN_FORCE_FILL_AND_KILL,
            price_type=PriceType.PRICE_TYPE_CURRENCY,
            confirm_margin_trade=False,
        )
        print(request)
        response: PostOrderAsyncResponse = await client.orders.post_order_async(request)
        client.orders_stream

        return response


new_order_response = await create_new_order()
new_order_response

PostOrderAsyncRequest(instrument_id='BBG0013HRTL0', quantity=1, price=Quotation(units=10, nano=836499999), direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, account_id='6def19bc-ad19-486f-92fa-090c72e531f0', order_type=<OrderType.ORDER_TYPE_MARKET: 2>, order_id='784fe7eb-f6e1-4f2b-83ac-91df54531082', time_in_force=<TimeInForceType.TIME_IN_FORCE_FILL_AND_KILL: 2>, price_type=<PriceType.PRICE_TYPE_CURRENCY: 2>, confirm_margin_trade=False)


PostOrderAsyncResponse(order_request_id='784fe7eb-f6e1-4f2b-83ac-91df54531082', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_NEW: 4>, trade_intent_id='cd9c53a1-e028-451d-aa96-eeb7133668fa')

In [ ]:
from tinkoff.invest.async_services import OrderIdType


async def order_status():
    async with AsyncSandboxClient(TOKEN) as client:
        # request = OrderStateStreamRequest([account_id])
        # async for msg in client.orders_stream.order_state_stream(request=request):
        #     print(msg)

        state = await client.orders.get_order_state(
            account_id=account_id,
            order_id="cb4702eb-d999-4510-afb4-ed1b88713036",
            order_id_type=OrderIdType.ORDER_ID_TYPE_REQUEST,
        )
        return state


state = await order_status()
state

OrderState(order_id='c4cabe45-b3aa-479e-99e1-c6071c6e14a3', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>, lots_requested=1, lots_executed=1, initial_order_price=MoneyValue(currency='rub', units=10837, nano=500000000), executed_order_price=MoneyValue(currency='rub', units=10837, nano=500000000), total_order_amount=MoneyValue(currency='rub', units=10837, nano=500000000), average_position_price=MoneyValue(currency='rub', units=10, nano=837500000), initial_commission=MoneyValue(currency='rub', units=5, nano=418750000), executed_commission=MoneyValue(currency='rub', units=5, nano=418750000), figi='BBG0013HRTL0', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=10, nano=837500000), stages=[], service_commission=MoneyValue(currency='rub', units=0, nano=0), currency='rub', order_type=<OrderType.ORDER_TYPE_MARKET: 2>, order_date=datetime.datetime(2025, 12, 9, 11, 4, 13, 757728, tzinfo=datetime.time

In [9]:
from pprint import pprint

pprint(state)

OrderState(order_id='c4cabe45-b3aa-479e-99e1-c6071c6e14a3',
           execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>,
           lots_requested=1,
           lots_executed=1,
           initial_order_price=MoneyValue(currency='rub',
                                          units=10837,
                                          nano=500000000),
           executed_order_price=MoneyValue(currency='rub',
                                           units=10837,
                                           nano=500000000),
           total_order_amount=MoneyValue(currency='rub',
                                         units=10837,
                                         nano=500000000),
           average_position_price=MoneyValue(currency='rub',
                                             units=10,
                                             nano=837500000),
           initial_commission=MoneyValue(currency='rub',
                                   

In [12]:
from tinkoff.invest.utils import quotation_to_decimal


async def get_account_balance():
    async with AsyncSandboxClient(TOKEN) as client:
        return (await client.operations.get_positions(account_id=account_id)).money
        


await get_account_balance()

[MoneyValue(currency='cny', units=5000, nano=0),
 MoneyValue(currency='rub', units=3945794, nano=410750000)]